The English description will be updated later.

本次可视化分析对于我而言是一个学习的过程，用到的工具是plotly。

我从[Dgomonov](https://www.kaggle.com/dgomonov)的[Advanced Fires Analysis with Plotly](https://www.kaggle.com/dgomonov/advanced-fires-analysis-with-plotly)，[Ethan Schacht](https://www.kaggle.com/etsc9287)的[A Quick Analysis of Forest Fires in Brazil](https://www.kaggle.com/etsc9287/a-quick-analysis-of-forest-fires-in-brazil)以及其他人的优秀作品中受到了很大的启发。

实验主要包含了每年火灾总数的散点图、每年火灾总数及每年每个季节火灾数占比的条形图、每年每月火灾数的盒图、各个州火灾总数的饼图，各个州每年火灾数量的热图以及各个月每年火灾数量的热图。

In [ ]:
import numpy as np
import pandas as pd
# 采用离线绘图方式
import plotly
import plotly.offline as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()

In [ ]:
# 读取数据，注意文件编码格式
df = pd.read_csv('../input/forest-fires-in-brazil/amazon.csv', encoding='ISO-8859-1')
df.head(10)

In [ ]:
# 检查数据集是否有需要舍弃的数据
print("共有%d条数据" % len(df))
print("空值检查")
print(df.isna().sum())

In [ ]:
# 概览所有统计到的州
df.state.unique()

In [ ]:
# 概览月份
df.month.unique()

In [ ]:
# 把月份转换成英语
month_map = {'Janeiro': 'January', 'Fevereiro': 'February', 'Março': 'March', 'Abril': 'April', 'Maio': 'May',
          'Junho': 'June', 'Julho': 'July', 'Agosto': 'August', 'Setembro': 'September', 'Outubro': 'October',
          'Novembro': 'November', 'Dezembro': 'December'}
df['month'] = df['month'].map(month_map)
df.month.unique()

In [ ]:
# 日期列没有意义，丢掉
df.drop(columns = ['date'], axis=1, inplace=True)
df.head(5)

In [ ]:
# 统计各年发生的火灾总数
years = list(df.year.unique())
sub_fires_per_year = []
for i in years:
    #数据集中汇报的火灾数有小数，四舍五入取整
    count = df.loc[df['year'] == i].number.sum().round(0)
    sub_fires_per_year.append(count)
fires_per_year_dic = {'year' : years, 'total_fires' : sub_fires_per_year}
print(fires_per_year_dic)

In [ ]:
# 绘制散点图
trace = go.Scatter(x = fires_per_year_dic['year'], y = fires_per_year_dic['total_fires'], mode = 'lines+markers')
layout = go.Layout(title='1998-2017年巴西火灾数量统计图', xaxis={'title':'年份'}, yaxis={'title':'数量'})
data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)

从1998年到2003年森林火灾数上升明显，从2003年到2008年森林火灾得到了比较好的控制数量有明显下降，但是之后一直到2017年每年火灾数起起伏伏，一直在35000起到42000起间。

In [ ]:
# 根据月份添加季节(巴西在南半球和北半球季节相反)
# 9、10、11月为春季，12、1、2月为夏季，3、4、5月为秋季，6、7、8月为冬季
season_map = {'January': 'Summer', 'February': 'Summer', 'March': 'Autumn', 'April': 'Autumn', 'May': 'Autumn',
              'June': 'Winter', 'July': 'Winter', 'August': 'Winter', 'September': 'Spring', 'October': 'Spring',
              'November': 'Spring', 'December': 'Summer'}
df['season'] = df['month'].map(season_map)
df.season.unique()

In [ ]:
# 把数据集按照年份和季节进行分割，并求出每年每月的火灾总数
def split_season(df, season):
    sub_fires = []
    for i in years:
        count = df.loc[(df['year'] == i) & (df['season'] == season)].number.sum().round(0)
        sub_fires.append(count)
    return sub_fires
sub_fires_in_spring = split_season(df, 'Spring')
sub_fires_in_summer = split_season(df, 'Summer')
sub_fires_in_autumn = split_season(df, 'Autumn')
sub_fires_in_winter = split_season(df, 'Winter')
print(sub_fires_in_spring)

In [ ]:
# 条形图绘制代码
trace_spring = go.Bar(y = years, x = sub_fires_in_spring,
                name = '春季', orientation = 'h', marker = dict(color = '#EF7A82'))
trace_summer = go.Bar(y = years, x = sub_fires_in_summer,
                name = '夏季', orientation = 'h', marker = dict(color = '#AFDD22'))
trace_autumn = go.Bar(y = years, x = sub_fires_in_autumn,
                name = '秋季', orientation = 'h', marker = dict(color = '#FFA631'))
trace_winter = go.Bar(y = years, x = sub_fires_in_winter,
                name = '冬季', orientation = 'h', marker = dict(color = '#44CEF6'))
data = [trace_spring, trace_summer, trace_autumn, trace_winter]
layout = go.Layout(
    title = '1998年-2017年各季节火灾数量及季节分布统计图',
    barmode='stack',
    xaxis={'title':'数量'},yaxis={'title':'年份'}
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

把每年发生的火灾总数绘制成条形图，并用不同的颜色表示不同季节发生的火灾。个颜色的长度代表着该季节森林火灾的数量，总长度代表着该年发生的森林火灾总数。

从图中我们不仅可以从条形图的角度看到每年火灾总数情况，还可以看出每年的火灾大都集中在了春季和冬季两个季节。


In [ ]:
# 把数据集按照season分割
fires_per_season = dict(list(df.groupby(df['season'])))
# 单独处理下12月，1月和2月，以便顺序展示月份
fires_per_season_summer1 = fires_per_season['Summer'].loc[fires_per_season['Summer']['month'].isin(['December'])]
fires_per_season_summer2 = fires_per_season['Summer'].loc[fires_per_season['Summer']['month'].isin(['January', 'February'])]
fires_per_season_summer = fires_per_season_summer1.append(fires_per_season_summer2)
#绘制箱图
trace_summer = go.Box(y=fires_per_season_summer.number, x=fires_per_season_summer.month,
                      name='Summer', boxpoints='all', marker = dict(color = '#AFDD22'))
trace_autumn = go.Box(y=fires_per_season['Autumn'].number, x=fires_per_season['Autumn'].month,
                      name='Autumn', boxpoints='all', marker = dict(color = '#FFA631'))
trace_winter = go.Box(y=fires_per_season['Winter'].number, x=fires_per_season['Winter'].month,
                      name='Winter', boxpoints='all', marker = dict(color = '#44CEF6'))
trace_spring = go.Box(y=fires_per_season['Spring'].number, x=fires_per_season['Spring'].month,
                      name='Spring', boxpoints='all', marker = dict(color = '#EF7A82'))
data = [trace_summer, trace_autumn, trace_winter, trace_spring]
layout = go.Layout(
    title = '1998年-2017年各季节火灾数量盒图',
    xaxis={'title':'月份'},yaxis={'title':'数量'}
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

把每个月分发生的森林火灾以箱图形式展现出来，通过该图我们可以有效的识别数据的特征，直观地识别统计数据中每个月报告的火灾数有无异常数量，以及判断历史上各月火灾数量的离散程度和偏向。

该图进一步证实了每年6月到12月是森林火灾发生的高峰期，在冬季和春季两个季节，月森林火灾数量超500的情况远超夏季和秋季。

In [ ]:
# 把数据集按照state进行分割，并求出每个地区的火灾总数
sub_fires_per_state = df['number'].groupby(df['state']).sum().round(0).sort_values(ascending=False)
# 绘制饼图
trace = go.Pie(labels=sub_fires_per_state.index, values=sub_fires_per_state.values, textinfo='label+value')
data = [trace]
layout = go.Layout(title = '1998年-2017年各州火灾数量饼图', height=800)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

把1998年到2017年各州的森林火灾数进行汇总绘制饼图，通过代表各州的扇面占整体的比例，可以清楚的看到各州森林火灾的发生情况。

其中排名前十的为：Mato Grosso、Paraiba、Sao Paulo、Rio、Bahia、Piau、Goias、Minas Gerais、Tocantins、Amazonas

本数据集的讨论中也提到[重复数据的问题](https://www.kaggle.com/gustavomodelli/forest-fires-in-brazil/discussion/113114)，这里我们同样认为他们是不同的州，因为不知道如何区分这些州，所以我仅用了数字作为标记，其他信息保持原样。

In [ ]:
# 有部分地区火灾数远超过其他地区，经过检查数据集中的地区有重名情况
# 数据集中Mato Grosso, Paraiba, Rio这三个地区有多个相同的名称，手动处理数据集进行标记后再重新进行探索
df2 = pd.read_csv('../input/forest-fires-in-brazil-relabeling/amazon_v2.csv', encoding='ISO-8859-1')
month_map = {'Janeiro': 'January', 'Fevereiro': 'February', 'Março': 'March', 'Abril': 'April', 'Maio': 'May',
          'Junho': 'June', 'Julho': 'July', 'Agosto': 'August', 'Setembro': 'September', 'Outubro': 'October',
          'Novembro': 'November', 'Dezembro': 'December'}
df2['month'] = df2['month'].map(month_map)
df2.drop(columns = ['date'], axis=1, inplace=True)
df2.head(5)

In [ ]:
# 把数据集按照新标记的state进行分割，并求出每个地区的火灾总数
sub_fires_per_state = df2['number'].groupby(df2['state']).sum().round(0).sort_values(ascending=False)
# 绘制饼图
trace = go.Pie(labels=sub_fires_per_state.index, values=sub_fires_per_state.values, textinfo='label+value')
data = [trace]
layout = go.Layout(title='1998年-2017年各州火灾数量饼图', height=800)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

数据集中存在相同名称州的统计信息，我们现在把这些州认为是不同的州，并对数据集手动进行标记。重新统计数据并绘制饼图，其中排名前十的为：Sao Paul、Mato Grosso1、Mato Grosso2、Bahia、Paraiba2、Piau、Goias、Minas Gerais、Tocantins、Amazonas

其中Mato Grossol州虽然被重新标记为两个州，但是它仍然排在森林火灾数量的第二名和第三名，而Paraiba排在第五名和二十一名，Rio被重新标记后则均未进入前三名。

In [ ]:
#根据州拆分数据集
fires_per_state = dict(list(df2.groupby(df2['state'])))
# print(fires_per_state)
states = df2['state'].unique()
years = df2['year'].unique()
fires_per_state_per_year = []
for state in states:
    sub_fires = fires_per_state[state]['number'].groupby(fires_per_state[state]['year']).sum().round(0)
    fires_per_state_per_year.append(list(sub_fires))
# print(fires_per_state_per_year)

In [ ]:
# 绘制热图
trace = go.Heatmap(z=fires_per_state_per_year, x=years, y=states, colorscale='reds')
data = [trace]
layout = go.Layout(title = '各州每年火灾数量热图', 
                   xaxis={'title':'年份', 'nticks':20},yaxis={'title':'州', 'nticks':27},
                   margin={'l':100})
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

把数据集中的数据按照州和年份进行汇总整理，绘制关于各州每年发生森林火灾数量的热图（HeatMap），通过颜色我们可以直观的了解各个州每年火灾数量的情况。

In [ ]:
months = df2['month'].unique()
# 月份如果为字符串的话将会按照字典顺序进行统计，所以手动转换为数值类型月份
num_month_map =  {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5,'June': 6,
                  'July': 7, 'August': 8, 'September': 9, 'October': 10,'November': 11, 'December': 12}
df2['month_num'] = df2['month'].map(num_month_map)
fires_per_state = dict(list(df2.groupby(df2['state'])))
# 按照月份分割数据集
fires_per_state_per_month = []
for state in states:
    sub_fires = fires_per_state[state]['number'].groupby(fires_per_state[state]['month_num']).sum().round(0)
    fires_per_state_per_month.append(list(sub_fires))
# print(fires_per_state_per_month)

In [ ]:
# 绘制热图
trace = go.Heatmap(z=fires_per_state_per_month, x=months, y=states, colorscale='reds')
data = [trace]
layout = go.Layout(title = '各州每月火灾数量热图', 
                   xaxis={'title':'月份', 'nticks':20},yaxis={'title':'州', 'nticks':27},
                   margin={'l':100})
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

各州每月发生森林火灾数量的热图（HeatMap），其中大多数州的火灾集中于下半年，但也有少数异常，比如Mato Grosso1州和Roraima州集中在上半年，Tocantins州则集中在年中等等。

## 总结
Plotly是一个数据可视化的利器，本次实验中对数据的处理后经过Plotly的动态可视化的展示，收获了很多关于巴西森林火灾的相关信息。

在过去的20年里，以2003年为时间点森林火灾的总数经历了先上升再下降的一个趋势，但是从2008年开始，每年森林火灾的总数未得到一个有效的控制，甚至还有一个增多的趋势。通过进一步对各个州的分析，我们发现Mato Grosso州森林火灾的问题尤为严重，它森林火灾的数量最多而且相对于其他州每年火灾的数量相对稳定的情况下，Mato Grosso州的火灾趋势是最不乐观的。

其次通过对不同月份的历史森林火灾数量的分析，我们可以看到冬季和春季的森林火灾最为严重，而且这两个季节历史月火灾数大于500的情况远大于夏季和秋季，所以应当在这些季节加强应对措施。当然也有如Roraima等州的森林火灾多发生于上半年，也就是夏季和秋季，所以应该结合实际制定适合自己的防范措施。
